In [1033]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # this is used for the plot the graph 
from datetime import date
import random

from sklearn.decomposition import PCA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dropout

import tensorflow as tf

%matplotlib inline

import random 
# random.seed(42)

from numpy.random import seed
seed(42)

In [1034]:
csv_path_1 = "training_dataset.csv"
csv_path_2 = "score.csv"

# 0. First, let's understand our data...

In [1035]:
df = pd.read_csv(csv_path_1) # load the pandas dataframe
df_score = pd.read_csv(csv_path_2)
initial_cols_to_drop = ["Unnamed: 0","Unnamed: 0.1", "period", "test", "recent_date", "date"] 
for col_name in initial_cols_to_drop: # drops columns that aren't supposed to be in dataset
    try:
        df = df.drop(columns=[col_name])
        df_score = df_score.drop(columns=[col_name])
    except:
        continue
display(df.head())

,dt,weekday,year,id_driver,id_carrier_number,dim_carrier_type,dim_carrier_company_name,home_base_city,home_base_state,carrier_trucks,...,most_recent_load_date,load_day,loads,marketplace_loads_otr,marketplace_loads_atlas,marketplace_loads,brokerage_loads_otr,brokerage_loads_atlas,brokerage_loads,total_loads
0,2019-12-16,Monday,2019,21350,U0109015,Owner Operator,CA&F TRUCKING,Maywood,CA,"[""poweronly""]",...,2021-02-17,2019-12-16,2,0,438,438,0,45,45,483
1,2021-01-15,Friday,2021,36437,C0097727,Fleet,New opportunities inc,Los Angeles,CA,"[""poweronly"", ""boxtruck""]",...,2021-02-03,2021-01-15,1,2,72,74,0,1,1,75
2,2019-12-26,Thursday,2019,19323,U0107081,Owner Operator,RAS,Compton,CA,"[""poweronly""]",...,2020-09-25,2019-12-26,1,0,180,180,0,2,2,182
3,2021-02-10,Wednesday,2021,34809,C0094651,Fleet,NFS asset Drayage,Lynwood,CA,"[""poweronly"", ""dryvan""]",...,2021-02-17,2021-02-10,3,0,0,0,0,0,0,62
4,2017-07-24,Monday,2017,4728,U0094376,Owner Operator,joes transportation,Norco,CA,"[""dryvan""]",...,2017-10-11,2017-07-24,2,57,0,57,314,0,314,371


In [1036]:
df.describe()

,year,id_driver,num_trucks,days_signup_to_approval,loads,marketplace_loads_otr,marketplace_loads_atlas,marketplace_loads,brokerage_loads_otr,brokerage_loads_atlas,brokerage_loads,total_loads
count,83414.000000,83414.000000,83344.000000,71124.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000
mean,2018.960930,18222.414954,22.582921,298.752489,2.076270,29.477762,71.579675,101.057436,148.258422,13.073021,161.331443,266.502661
std,1.359343,11667.704926,48.829719,390.345107,2.672163,88.171940,194.532776,214.502147,415.978060,42.241592,413.792137,448.806175
min,2015.000000,20.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2018.000000,7890.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,37.000000
50%,2019.000000,16299.000000,4.000000,61.000000,1.000000,2.000000,0.000000,13.000000,15.000000,0.000000,37.000000,110.000000
75%,2020.000000,28974.000000,14.000000,497.000000,2.000000,23.000000,18.000000,94.000000,112.000000,1.000000,135.000000,325.000000
max,2021.000000,38125.000000,195.000000,1653.000000,129.000000,902.000000,1324.000000,1348.000000,4266.000000,371.000000,4266.000000,4266.000000


In [1037]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83414 entries, 0 to 83413
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   dt                        83414 non-null  object 
 1   weekday                   83414 non-null  object 
 2   year                      83414 non-null  int64  
 3   id_driver                 83414 non-null  int64  
 4   id_carrier_number         83414 non-null  object 
 5   dim_carrier_type          83414 non-null  object 
 6   dim_carrier_company_name  83365 non-null  object 
 7   home_base_city            83369 non-null  object 
 8   home_base_state           83369 non-null  object 
 9   carrier_trucks            83414 non-null  object 
 10  num_trucks                83344 non-null  float64
 11  interested_in_drayage     83414 non-null  object 
 12  port_qualified            83414 non-null  object 
 13  signup_source             83414 non-null  object 
 14  ts_sig

# 1. Generate Labels

In [1038]:
# converts date from csv to a python datetime object making it easier to work with
def convert_dates(df):
    dates_columns = ['most_recent_load_date', 'first_load_date', 'load_day', 'dt']
    for col_name in dates_columns:
        try:
            df[col_name] = pd.to_datetime(df[col_name], format='%Y-%m-%d')
        except:
            continue

convert_dates(df)
convert_dates(df_score)

In [1039]:
df['most_recent_load_date'].head()

0   2021-02-17
1   2021-02-03
2   2020-09-25
3   2021-02-17
4   2017-10-11
Name: most_recent_load_date, dtype: datetime64[ns]

In [1040]:
total_loads75 = df.total_loads.quantile(0.75) # finds 75th percentile of loads
most_recent_load_date75 = df.most_recent_load_date.quantile(0.75) # finds 75th percentile of most recent load date

print(total_loads75)
print(most_recent_load_date75)
# Manual Check
# sorted_dts = sorted(list(df.most_recent_load_date))
# quartile_estimate_index = int(len(sorted_dts)*0.75)
# print("SORTED INDEX", sorted_dts[quartile_estimate_index])

325.0
2021-02-14 00:00:00


In [1041]:
# new_labels = {"label": {}}
num_days_worked_dict = {}

for index, row in df.iterrows(): # changes the labels in the label columns
    # checks if the load and most recent load date are in the 75th percentile
    if row["total_loads"] >= total_loads75 and row["most_recent_load_date"] >= most_recent_load_date75:
        df.at[index, "label"] = 1
    else:
        df.at[index, "label"] = 0

In [1042]:
uniqueValues, occurCount = np.unique(np.array(df["label"]), return_counts=True)
print("Unique Values : " , uniqueValues)
print("Occurrence Count : ", occurCount)

Unique Values :  [0. 1.]
Occurrence Count :  [73021 10393]


In [1043]:
loc_arr = []
for index, row in df.iterrows():
    if pd.isnull(row["home_base_city"]) != True and pd.isnull(row["home_base_state"]) != True:
        if (row["home_base_city"], row["home_base_state"]) not in loc_arr:
            loc_arr.append((row["home_base_city"], row["home_base_state"]))

In [1044]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83414 entries, 0 to 83413
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   dt                        83414 non-null  datetime64[ns]
 1   weekday                   83414 non-null  object        
 2   year                      83414 non-null  int64         
 3   id_driver                 83414 non-null  int64         
 4   id_carrier_number         83414 non-null  object        
 5   dim_carrier_type          83414 non-null  object        
 6   dim_carrier_company_name  83365 non-null  object        
 7   home_base_city            83369 non-null  object        
 8   home_base_state           83369 non-null  object        
 9   carrier_trucks            83414 non-null  object        
 10  num_trucks                83344 non-null  float64       
 11  interested_in_drayage     83414 non-null  object        
 12  port_qualified    

In [1045]:
groups = df.groupby("id_driver")

In [1046]:
# NO: dt, weekday, year, id_carrier_number, dim_preferred_lanes, load_day, loads
new_arr = []
for key, group in groups:
    group.sort_values(by="load_day", ascending=False, inplace=True)
    temp_arr = []
    temp_arr.append(key)
    
    if group["dim_carrier_type"].nunique() == 2:
        temp_arr.append("Both")
    elif group["dim_carrier_type"].nunique() == 0:
        temp_arr.append(None)
    else:
        temp_arr.append((group["dim_carrier_type"].iloc[0]))
    
    
    idxmax_cols = ["dim_carrier_company_name", 
                   "carrier_trucks", "signup_source", "ts_signup", "ts_first_approved",
                  "days_signup_to_approval"] #"home_base_city", "home_base_state",
    
    for col in idxmax_cols:
        try:
            temp_arr.append(group[col].value_counts().dropna(how="any").idxmax())
        except:
            
            temp_arr.append(None)
    
    try:
        x = len(group["home_base_city"].dropna(how="any").iloc[0])
        x = len(group["home_base_state"].dropna(how="any").iloc[0])
        temp_arr.append(group["home_base_city"].dropna(how="any").iloc[0])
        temp_arr.append(group["home_base_state"].dropna(how="any").iloc[0])
    except:
        index_num = random.randint(0, len(loc_arr)-1)
        temp_arr.append(loc_arr[index_num][0])
        temp_arr.append(loc_arr[index_num][1])
        
    
    
    try:
        temp_arr.append(group["num_trucks"].dropna(how="any").mean())
    except:
        temp_arr.append(None)
        
    iloc_cols = ["interested_in_drayage", "port_qualified", "driver_with_twic", 
                 "first_load_date", "most_recent_load_date", "marketplace_loads_otr", 
                 "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
                 "brokerage_loads_atlas", "brokerage_loads", "total_loads"]
    for col in iloc_cols:
        try:
            temp_arr.append(group[col].dropna(how="any").iloc[0])
        except:
            temp_arr.append(None)
    
    temp_arr.append(group.shape[0])
    
    temp_arr.append(((pd.to_datetime(date.today()) - group["most_recent_load_date"].dropna(how="any").iloc[0]).days) * (-1))
    
    temp_arr.append(group["label"].value_counts().dropna(how="any").idxmax())
    
    new_arr.append(np.array(temp_arr))

In [1047]:
column_names = ["id_driver", "dim_carrier_type", "dim_carrier_company_name", 
                "carrier_trucks", "signup_source", "ts_signup", "ts_first_approved",
                "days_signup_to_approval", "home_base_city", "home_base_state", "num_trucks", 
                "interested_in_drayage", "port_qualified", "driver_with_twic", 
                "first_load_date", "most_recent_load_date", "marketplace_loads_otr", 
                "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
                "brokerage_loads_atlas", "brokerage_loads", "total_loads", "num_trips_made", 
                "days_since_last_load", "label"]

df = pd.DataFrame(np.array(new_arr), columns=column_names)

In [1048]:
for col in column_names:
    try:
        df[col] = df[col].convert_dtypes()
    except:
        continue

In [1049]:
# # NO: dt, weekday, year, id_carrier_number, dim_preferred_lanes, load_day, loads
new_dict = {}
label_dict = {}
most_recent_date_arr = []
num_trips_arr = []
for key, group in groups:
    group.sort_values(by="load_day", ascending=False, inplace=True)
    if key not in new_dict:
        try:
            new_dict[key] = (group["load_day"].dropna(how="any").iloc[0], group.shape[0])
        except:
            new_dict[key] = None
        
        try:
            label_dict[key] = group["label"].dropna(how="any").iloc[0]
        except:
            label_dict[key] = 1

In [1050]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5291 entries, 0 to 5290
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id_driver                 5291 non-null   Int64         
 1   dim_carrier_type          5291 non-null   string        
 2   dim_carrier_company_name  5284 non-null   string        
 3   carrier_trucks            5291 non-null   string        
 4   signup_source             5291 non-null   string        
 5   ts_signup                 5291 non-null   string        
 6   ts_first_approved         3962 non-null   string        
 7   days_signup_to_approval   3962 non-null   Int64         
 8   home_base_city            5291 non-null   string        
 9   home_base_state           5291 non-null   string        
 10  num_trucks                5249 non-null   float64       
 11  interested_in_drayage     5291 non-null   string        
 12  port_qualified      

In [1051]:
df["label"].describe()

count    5291.000000
mean        0.010395
std         0.101434
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: label, dtype: float64

In [1052]:
groups = df_score.groupby("id_driver")

In [1053]:
# # NO: dt, weekday, year, id_carrier_number, dim_preferred_lanes, load_day, loads
new_dict = {}
most_recent_date_arr = []
num_trips_arr = []
score_labels = []
days_since_last_work_arr = []
trips_dict = {}
for key, group in groups:
    group.sort_values(by="load_day", ascending=False, inplace=True)
    if key not in new_dict:
        try:
            new_dict[key] = (group["load_day"].dropna(how="any").iloc[0], group.shape[0])
        except:
            new_dict[key] = None
    if key not in trips_dict:
        trips_dict[key] = group.shape[0]

for index, row in df_score.iterrows():
    most_recent_date_arr.append(new_dict[row["id_driver"]][0])
    num_trips_arr.append(new_dict[row["id_driver"]][1])
    if row["id_driver"] in label_dict:
        score_labels.append(label_dict[row["id_driver"]])
    else:
        print("Not here: ", row["id_driver"])
        rand_num = random.randint(0, 1000)
        if rand_num <= 125:
            score_labels.append(1)
            label_dict[row["id_driver"]] = 1
        else:
            score_labels.append(0)
            label_dict[row["id_driver"]] = 0
            
    try:
        x = len(row["home_base_city"])
        x = len(row["home_base_state"])
    except:
        index_num = random.randint(0, len(loc_arr)-1)
        df_score.at[index, "home_base_city"] = loc_arr[index_num][0]
        df_score.at[index, "home_base_state"] = loc_arr[index_num][1]
        
    days_since_last_work_arr.append(((pd.to_datetime(date.today()) - row["load_day"]).days) * (-1))

score_labels = np.array(score_labels)
df_score["most_recent_load_date"] = np.array(most_recent_date_arr)
df_score["num_trips_made"] = np.array(num_trips_arr)
df_score["days_since_last_load"] = np.array(days_since_last_work_arr)

Not here:  13711
Not here:  31557
Not here:  16642
Not here:  13761
Not here:  8205
Not here:  35572
Not here:  7623
Not here:  31873
Not here:  12429
Not here:  1288
Not here:  29653
Not here:  20357
Not here:  15073
Not here:  29070
Not here:  10359
Not here:  11514
Not here:  35563
Not here:  7594
Not here:  26545


In [1054]:
column_names = ["id_driver", "dim_carrier_type", "dim_carrier_company_name", 
                "carrier_trucks", "signup_source", "ts_signup", "ts_first_approved",
                "days_signup_to_approval", "home_base_city", "home_base_state", "num_trucks", 
                "interested_in_drayage", "port_qualified", "driver_with_twic", 
                "first_load_date", "most_recent_load_date", "marketplace_loads_otr", 
                "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
                "brokerage_loads_atlas", "brokerage_loads", "num_trips_made", 
                "days_since_last_load"]

df_temp = df_score.drop(columns = column_names)
to_drop = list(df_temp.columns.values)
df_score = df_score.drop(columns=to_drop)
df_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id_driver                 1000 non-null   int64         
 1   dim_carrier_type          1000 non-null   object        
 2   dim_carrier_company_name  1000 non-null   object        
 3   home_base_city            1000 non-null   object        
 4   home_base_state           1000 non-null   object        
 5   carrier_trucks            1000 non-null   object        
 6   num_trucks                1000 non-null   float64       
 7   interested_in_drayage     1000 non-null   object        
 8   port_qualified            1000 non-null   object        
 9   signup_source             1000 non-null   object        
 10  ts_signup                 1000 non-null   object        
 11  ts_first_approved         854 non-null    object        
 12  days_signup_to_approv

In [1055]:
for col in column_names:
    try:
        df_score[col] = df_score[col].convert_dtypes()
    except:
        continue

In [1056]:
display(df_score["num_trips_made"])

0      4
1      1
2      1
3      3
4      1
      ..
995    1
996    3
997    1
998    1
999    1
Name: num_trips_made, Length: 1000, dtype: Int64

In [1057]:
df_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id_driver                 1000 non-null   Int64         
 1   dim_carrier_type          1000 non-null   string        
 2   dim_carrier_company_name  1000 non-null   string        
 3   home_base_city            1000 non-null   string        
 4   home_base_state           1000 non-null   string        
 5   carrier_trucks            1000 non-null   string        
 6   num_trucks                1000 non-null   Int64         
 7   interested_in_drayage     1000 non-null   string        
 8   port_qualified            1000 non-null   string        
 9   signup_source             1000 non-null   string        
 10  ts_signup                 1000 non-null   string        
 11  ts_first_approved         854 non-null    string        
 12  days_signup_to_approv

In [1058]:
drop_cols = ["dt", "weekday", "year", "id_carrier_number", "dim_preferred_lanes", "load_day", "loads"]
for col in drop_cols:
    try:
        df_score = df_score.drop(columns=[col])
    except:
        continue

for col in drop_cols:
    try:
        df = df.drop(columns=[col])
    except:
        continue

In [1059]:
df_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id_driver                 1000 non-null   Int64         
 1   dim_carrier_type          1000 non-null   string        
 2   dim_carrier_company_name  1000 non-null   string        
 3   home_base_city            1000 non-null   string        
 4   home_base_state           1000 non-null   string        
 5   carrier_trucks            1000 non-null   string        
 6   num_trucks                1000 non-null   Int64         
 7   interested_in_drayage     1000 non-null   string        
 8   port_qualified            1000 non-null   string        
 9   signup_source             1000 non-null   string        
 10  ts_signup                 1000 non-null   string        
 11  ts_first_approved         854 non-null    string        
 12  days_signup_to_approv

# 3. Basic Statistics

In [1060]:
corr_matrix = df.corr()
for col_name in (list(df.columns.values)): # prints all the correlation matrices corresponding to each feature
    try:
        print(col_name)
        display(corr_matrix[col_name].sort_values(ascending=False))
        print('---------------------------------------------------------------------')
    except:
        print("{} is not of type integer".format(col_name))
        print('---------------------------------------------------------------------')

id_driver


id_driver                  1.000000
days_since_last_load       0.839446
num_trucks                 0.169567
brokerage_loads_atlas      0.145307
marketplace_loads_atlas    0.138116
marketplace_loads          0.123344
label                      0.075683
total_loads                0.052722
num_trips_made             0.018007
marketplace_loads_otr     -0.012441
brokerage_loads           -0.037352
brokerage_loads_otr       -0.063795
days_signup_to_approval   -0.734778
Name: id_driver, dtype: float64

---------------------------------------------------------------------
dim_carrier_type
dim_carrier_type is not of type integer
---------------------------------------------------------------------
dim_carrier_company_name
dim_carrier_company_name is not of type integer
---------------------------------------------------------------------
carrier_trucks
carrier_trucks is not of type integer
---------------------------------------------------------------------
signup_source
signup_source is not of type integer
---------------------------------------------------------------------
ts_signup
ts_signup is not of type integer
---------------------------------------------------------------------
ts_first_approved
ts_first_approved is not of type integer
---------------------------------------------------------------------
days_signup_to_approval


days_signup_to_approval    1.000000
brokerage_loads_otr        0.048116
brokerage_loads            0.027929
num_trips_made             0.010462
marketplace_loads_otr      0.007436
total_loads               -0.043765
num_trucks                -0.053498
label                     -0.064084
marketplace_loads         -0.089268
brokerage_loads_atlas     -0.092088
marketplace_loads_atlas   -0.099058
days_since_last_load      -0.628732
id_driver                 -0.734778
Name: days_signup_to_approval, dtype: float64

---------------------------------------------------------------------
home_base_city
home_base_city is not of type integer
---------------------------------------------------------------------
home_base_state
home_base_state is not of type integer
---------------------------------------------------------------------
num_trucks


...

---------------------------------------------------------------------
days_since_last_load


days_since_last_load       1.000000
id_driver                  0.839446
num_trips_made             0.284921
marketplace_loads          0.236472
total_loads                0.216076
marketplace_loads_atlas    0.202243
brokerage_loads_atlas      0.179735
num_trucks                 0.163449
marketplace_loads_otr      0.157085
label                      0.142288
brokerage_loads            0.092076
brokerage_loads_otr        0.061265
days_signup_to_approval   -0.628732
Name: days_since_last_load, dtype: float64

---------------------------------------------------------------------
label


label                      1.000000
marketplace_loads          0.584442
marketplace_loads_atlas    0.577596
total_loads                0.541811
num_trips_made             0.481593
brokerage_loads_atlas      0.327819
brokerage_loads            0.293505
brokerage_loads_otr        0.239032
marketplace_loads_otr      0.163324
days_since_last_load       0.142288
id_driver                  0.075683
num_trucks                -0.035292
days_signup_to_approval   -0.064084
Name: label, dtype: float64

---------------------------------------------------------------------


 Also year and TODO_FIND_COLUMN_NAME_2 and year are highly correlated and have a similar impact on label, so we could drop one? 

Is there really a need for brokerage_loads when it is so highly correlated to brokerage_loads_otr due to the vast majority of shipments being delivered over-the-road as compared to via ATLAS? 

I have the same question about total_loads due to the vast majority of loads being brokerage loads...

What's the point of having both year and date?

We can remove the id_carrier_number column from this dataset as it is not relevant to predicting a label of 0 or 1 (When trying to find high performing drivers, we need to know their carrier number, so we can extract the id_carrier_number column for now...)

We could one-hot-encode sign-up source and see its effect on labels.

We can remove the ts_first_approved column because the date of approval shouldn't matter that much but instead the days_signup_to_approval matter.

dim_preferred_lanes only has a few values so we can either remove the column or impute values.

Also first_load_date, most_recent_load_date and load_day shouldn't matter much. Instead we can have values such as: number of days doing the job = most_recent_load_date - first_load_date
AND
days_from_last_load_to_today = todays_date - most_recent_load_date

There are also a couple other features we need to impute.

Also, only people that are port qualified can provide drayage services, so we should create a field called qualified_and_interest_in_drayage which is only 1 (yes) when interested_in_drayage = "yes" and port_qualified = "yes". We can also cross these features...

# 4. Data Feature Extraction Plan and Pipeline

In [1061]:
def drayage_feature_cross(df):
    loc_cross = list(zip(df["home_base_city"], df["home_base_state"]))# feature cross to get (city, state) tuple
    # feature cross for interested in drayage and port qualified
    drayage_cross = list(zip(df["interested_in_drayage"], df["port_qualified"]))

    drayage_arr = []
    for list_item in drayage_cross:
        if list_item[0] == "yes" and list_item[1] == "yes":
            drayage_arr.append("000001")
        if list_item[0] == "yes" and list_item[1] == "no":
            drayage_arr.append("000010")
        if list_item[0] == "no" and list_item[1] == "yes":
            drayage_arr.append("000100")
        if list_item[0] == "no" and list_item[1] == "no":
            drayage_arr.append("001000")
        if list_item[0] == "not specified" and list_item[1] == "yes":
            drayage_arr.append("010000")
        if list_item[0] == "not specified" and list_item[1] == "no":
            drayage_arr.append("100000")

    df["drayage_interested_port_qualified"] = np.array(drayage_arr)
    display(df["drayage_interested_port_qualified"])

drayage_feature_cross(df)
drayage_feature_cross(df_score)

0       100000
1       100000
2       100000
3       100000
4       100000
         ...  
5286    100000
5287    000001
5288    000001
5289    000001
5290    000001
Name: drayage_interested_port_qualified, Length: 5291, dtype: object

0      100000
1      000010
2      100000
3      000010
4      000010
        ...  
995    100000
996    100000
997    100000
998    100000
999    000010
Name: drayage_interested_port_qualified, Length: 1000, dtype: object

In [1062]:
id_driver_number_col = np.array(df["id_driver"]) # extract id_driver column
id_driver_number_col_score = np.array(df_score["id_driver"]) # extract id_driver column

drop_cols = ["id_driver", "interested_in_drayage", "port_qualified", 
             "ts_signup", "ts_first_approved"] #"home_base_city","home_base_state",
for col in drop_cols:
    try:
        df_score = df_score.drop(columns=[col])
    except:
        continue

for col in drop_cols:
    try:
        df = df.drop(columns=[col])
    except:
        continue

In [1063]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5291 entries, 0 to 5290
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   dim_carrier_type                   5291 non-null   string        
 1   dim_carrier_company_name           5284 non-null   string        
 2   carrier_trucks                     5291 non-null   string        
 3   signup_source                      5291 non-null   string        
 4   days_signup_to_approval            3962 non-null   Int64         
 5   home_base_city                     5291 non-null   string        
 6   home_base_state                    5291 non-null   string        
 7   num_trucks                         5249 non-null   float64       
 8   driver_with_twic                   5291 non-null   string        
 9   first_load_date                    5291 non-null   datetime64[ns]
 10  most_recent_load_date              5

In [1064]:
names = {}
for index, row in df.iterrows():
    names[row["dim_carrier_company_name"]] = int(names.get(row["dim_carrier_company_name"], 0) + 1)
listo = list(names.items())
listo.sort(reverse=True, key=lambda x: int(x[1]))
#listo[:50]

In [1065]:
listo = listo[:50]
count_50 = sum([x[1] for x in listo])
print("# 50: ", count_50)
print("Percentage 50: ", count_50/len(names))

names_arr = [tuples[0] for tuples in listo]
print(names_arr)

# 50:  1290
Percentage 50:  0.5176565008025682
['NFS asset Drayage', 'MC Express Trucking LLC', 'Roadrunner Transportation', 'Dong Fang Marketing Inc', 'ROADMOND LOGISTICS INC.', 'Consistent Trucking Inc', 'BLUE FREIGHT TRANSPORT INC', 'Convoy Express', 'Mega Fleet', 'iDC Drayage', 'Cross World Logistics', 'USA Diamonds Trucking', 'pointdirect', 'Chaidez Trucking', 'Saia LTL Freight', 'American Better Choice Corporation', 'American Freightways Lp.', 'Fastrucking', 'MERIDIAN LOGISTICS INC', 'Carlos Flores', 'J&G Transportation Group Inc', 'KLF transport inc', 'Star Rain LLC', 'Starco Logistics Inc.', '664 Transport', 'FTS EXPRESS INC', 'IDC OTR', "Luna's Transportation group", 'AGRAMONT TRANSPORT INC', 'BGood VirtueT Inc', 'Geber Freight', 'R&Y Castellanos Trucking Inc.', 'JC Transport', 'Great Qin Transportation LLC', 'JM Express Inc', 'MT Brothers Groups', 'cbt trucking', 'Road Eagle Logistics Corp', 'O.A. EXPRESS INC', 'AMPAK Logistics INC.', 'Kuang Trucking Inc.', 'nolan transportat

In [1066]:
def bucketize(df):
    days_worked = []
    for index, row in df.iterrows(): # bucketize the most frequent dim_carrier_company names, 
                                     # put less frequent names in a single bucket
        try:
            if row["dim_carrier_company_name"] not in names_arr:
                df.at[index, "dim_carrier_company_name"] = "Other"
        except:
            df.at[index, "dim_carrier_company_name"] = "Other"

        # find number of days driver has worked
        if row["most_recent_load_date"] != np.nan and row["first_load_date"] != np.nan:
            days_worked.append((row["most_recent_load_date"] - row["first_load_date"]).days)
        else:
            days_worked.append(None)
    df["days_tenured"] = np.array(days_worked)

bucketize(df)
bucketize(df_score)

In [1067]:
drop_cols = ["most_recent_load_date", "first_load_date", "weekday", "load_day", "total_loads"]
for col in drop_cols:
    try:
        df_score = df_score.drop(columns=[col])
    except:
        continue

for col in drop_cols:
    try:
        df = df.drop(columns=[col])
    except:
        continue

In [1068]:
df_unlabeled = df.drop(columns=["label"])
labels = df["label"].copy()

In [1069]:
uniqueValues, occurCount = np.unique(labels, return_counts=True)
print("Unique Values : " , uniqueValues)
print("Occurrence Count : ", occurCount)

Unique Values :  [0 1]
Occurrence Count :  [5236   55]


In [1070]:
df_unlabeled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5291 entries, 0 to 5290
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   dim_carrier_type                   5291 non-null   string 
 1   dim_carrier_company_name           5291 non-null   string 
 2   carrier_trucks                     5291 non-null   string 
 3   signup_source                      5291 non-null   string 
 4   days_signup_to_approval            3962 non-null   Int64  
 5   home_base_city                     5291 non-null   string 
 6   home_base_state                    5291 non-null   string 
 7   num_trucks                         5249 non-null   float64
 8   driver_with_twic                   5291 non-null   string 
 9   marketplace_loads_otr              5291 non-null   Int64  
 10  marketplace_loads_atlas            5291 non-null   Int64  
 11  marketplace_loads                  5291 non-null   Int64

In [1071]:
df_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   dim_carrier_type                   1000 non-null   string
 1   dim_carrier_company_name           1000 non-null   string
 2   home_base_city                     1000 non-null   string
 3   home_base_state                    1000 non-null   string
 4   carrier_trucks                     1000 non-null   string
 5   num_trucks                         1000 non-null   Int64 
 6   signup_source                      1000 non-null   string
 7   days_signup_to_approval            854 non-null    Int64 
 8   driver_with_twic                   1000 non-null   string
 9   marketplace_loads_otr              1000 non-null   Int64 
 10  marketplace_loads_atlas            1000 non-null   Int64 
 11  marketplace_loads                  1000 non-null   Int64 
 12  brokera

In [1072]:
cols = list(df_unlabeled.columns.values)
print(cols)

['dim_carrier_type', 'dim_carrier_company_name', 'carrier_trucks', 'signup_source', 'days_signup_to_approval', 'home_base_city', 'home_base_state', 'num_trucks', 'driver_with_twic', 'marketplace_loads_otr', 'marketplace_loads_atlas', 'marketplace_loads', 'brokerage_loads_otr', 'brokerage_loads_atlas', 'brokerage_loads', 'num_trips_made', 'days_since_last_load', 'drayage_interested_port_qualified', 'days_tenured']


In [1073]:
df_score = df_score[cols]

df_score['num_trucks'] = pd.to_numeric(df_score['num_trucks'], errors='coerce')
df_score['days_signup_to_approval'] = pd.to_numeric(df_score['days_signup_to_approval'], errors='coerce')

df_unlabeled['num_trucks'] = pd.to_numeric(df_unlabeled['num_trucks'], errors='coerce')
df_unlabeled['days_signup_to_approval'] = pd.to_numeric(df_unlabeled['days_signup_to_approval'], errors='coerce')

convert = ["id_driver", "days_signup_to_approval", "marketplace_loads_otr", 
               "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
               "brokerage_loads_atlas", "brokerage_loads", "num_trips_made",
               "num_trucks", "dim_carrier_type", "dim_carrier_company_name",
               "interested_in_drayage", "port_qualified", "signup_source", "driver_with_twic",
          "home_base_city","home_base_state"] #"home_base_city","home_base_state"
for col in convert:
    try:
        df_score[col] = df_score[col].convert_dtypes()
    except:
        continue

for col in convert:
    try:
        df_unlabeled[col] = df_unlabeled[col].convert_dtypes()
    except:
        continue

In [1074]:
df_concat = pd.concat([df_unlabeled, df_score], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6291 entries, 0 to 6290
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   dim_carrier_type                   6291 non-null   string
 1   dim_carrier_company_name           6291 non-null   string
 2   carrier_trucks                     6291 non-null   string
 3   signup_source                      6291 non-null   string
 4   days_signup_to_approval            4816 non-null   Int64 
 5   home_base_city                     6291 non-null   string
 6   home_base_state                    6291 non-null   string
 7   num_trucks                         6249 non-null   object
 8   driver_with_twic                   6291 non-null   string
 9   marketplace_loads_otr              6291 non-null   Int64 
 10  marketplace_loads_atlas            6291 non-null   Int64 
 11  marketplace_loads                  6291 non-null   Int64 
 12  broker

In [1075]:
for col in convert:
    try:
        df_concat[col] = df_concat[col].convert_dtypes()
    except:
        continue

In [1076]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6291 entries, 0 to 6290
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   dim_carrier_type                   6291 non-null   string 
 1   dim_carrier_company_name           6291 non-null   string 
 2   carrier_trucks                     6291 non-null   string 
 3   signup_source                      6291 non-null   string 
 4   days_signup_to_approval            4816 non-null   Int64  
 5   home_base_city                     6291 non-null   string 
 6   home_base_state                    6291 non-null   string 
 7   num_trucks                         6249 non-null   float64
 8   driver_with_twic                   6291 non-null   string 
 9   marketplace_loads_otr              6291 non-null   Int64  
 10  marketplace_loads_atlas            6291 non-null   Int64  
 11  marketplace_loads                  6291 non-null   Int64

In [1077]:
imputer = IterativeImputer()
categorical_features_one_hot = ["dim_carrier_type", "dim_carrier_company_name", "carrier_trucks", 
                                "signup_source", "driver_with_twic", "home_base_city","home_base_state"] #"home_base_city","home_base_state"

df_num = df_concat.drop(columns=categorical_features_one_hot)
numerical_features = list(df_num)

num_pipeline = Pipeline([
        ('imputer', imputer),
        ('std_scaler', StandardScaler()),
    ])

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, numerical_features),
        ("cat", OneHotEncoder(sparse=False), categorical_features_one_hot), #sparse=False
    ])
df_prepared = full_pipeline.fit_transform(df_concat)

In [1078]:
display(df_prepared)

array([[ 1.85116313, -0.39965744, -0.20109732, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.87262531, -0.39965744, -0.20109732, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.88612658, -0.39965744, -0.20109732, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.07263803, -0.39965744,  0.23165021, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.75297732,  1.95624695, -0.20109732, ...,  0.        ,
         0.        ,  0.        ],
       [-1.00674481, -0.18762604, -0.10999258, ...,  0.        ,
         0.        ,  0.        ]])

In [1079]:
X = df_prepared[:5291]
y = labels
X_test_score = df_prepared[5291:]
y_test_score = score_labels
y_test_score = np.array(y_test_score)
y = y.astype('int')
X, y = SMOTE().fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [1080]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)
print(X_test_score.shape, y_test_score.shape)

(8377, 647) (2095, 647)
(8377,) (2095,)
(1000, 647) (1000,)


# 5. Linear Regression

In [1081]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
lr_predicted = lin_reg.predict(X_test)

In [1082]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, lr_predicted)
rmse = np.sqrt(mse)
rmse

20480946.84227338

In [1083]:
from sklearn.metrics import r2_score

r2_score(y_test, lr_predicted)

-1678155469327497.0

In [1084]:
import statsmodels.api as sm

X_new = sm.add_constant(X_train)
toyregr_sm = sm.OLS(y_train.astype(float), X_new.astype(float))
results_sm = toyregr_sm.fit()

print(results_sm.summary())

                            OLS Regression Results                            
Dep. Variable:                  label   R-squared:                       0.904
Model:                            OLS   Adj. R-squared:                  0.897
Method:                 Least Squares   F-statistic:                     137.9
Date:                Thu, 18 Mar 2021   Prob (F-statistic):               0.00
Time:                        16:18:02   Log-Likelihood:                 3724.1
No. Observations:                8377   AIC:                            -6378.
Df Residuals:                    7842   BIC:                            -2616.
Df Model:                         534                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0002      0.025     -0.008      0.9

In [1085]:
from sklearn.linear_model import LogisticRegression

# y_temp = y_train.astype('int')

log_reg = LogisticRegression(max_iter=1000, multi_class='ovr')
log_reg.fit(X_train, y_train)
log_predicted = log_reg.predict(X_test)

In [1086]:
mse = mean_squared_error(y_test, log_predicted)
rmse = np.sqrt(mse)
rmse

0.06554344147787575

In [1087]:
r2_score(y_test, log_predicted)

0.9828133744852215

In [1088]:
from sklearn.metrics import confusion_matrix

accuracy = (accuracy_score(y_test, log_predicted))
print("accuracy = {}%".format(accuracy))
print("roc_auc_score = {}".format(roc_auc_score(y_test, log_predicted)))
print("F1-score = {}".format(f1_score(y_test, log_predicted)))

confusion_matrix(y_test, log_predicted)

accuracy = 0.9957040572792363%
roc_auc_score = 0.9957587181903864
F1-score = 0.9956668271545499


array([[1052,    9],
       [   0, 1034]])

# 6. PCA

In [1089]:
pca = PCA(n_components=0.95) # Create an instance of PCA model
pca.fit(X_train) # Fit X_train to PCA
X_train = pca.transform(X_train) # transform training data
X_test = pca.transform(X_test) # transform test data
X_test_score_new = pca.transform(X_test_score) # transform score test data
print(pca.explained_variance_)
print(pca.n_components_)

[23.02403235 14.8292382   4.7524796   3.43137662  1.34095534  0.91074499
  0.64874155  0.4554143 ]
8


In [1090]:
print(X_train)

[[-4.56072222 -0.57757824 -0.18739078 ...  0.37283851 -0.48076528
   0.21533659]
 [ 5.37456543  6.39873191 -6.11833163 ...  1.09087228  0.66087737
  -0.20263969]
 [ 2.88971566  8.39506031 -0.90284581 ... -2.09916304 -1.37083757
   0.9057837 ]
 ...
 [ 1.63578445 -1.82888682 -0.4267057  ... -0.13569799 -0.05801183
  -0.32058679]
 [-4.19701409  0.18815678 -0.38575999 ...  0.15592229 -0.03875752
  -0.03139817]
 [ 3.8415452  -1.82514819  9.13115583 ...  0.3777867   0.6996488
  -0.26273237]]


In [1091]:
print(X_test)

[[ 2.81515529 -2.39867401  5.66542471 ...  0.36900056 -0.1246603
   0.07862968]
 [ 2.84188191 -1.89420948  0.10792595 ... -0.28184803 -0.80357005
   0.30222688]
 [-4.6407357  -0.56059659 -0.21440874 ...  0.35191414  0.08350152
   0.46859202]
 ...
 [-3.60017128  0.77364615 -0.59388333 ... -0.22186177 -0.07029444
  -0.15169715]
 [ 3.66627559 -2.98059592  2.46828587 ...  0.43545875 -0.28098249
   0.13599173]
 [ 2.7714826   4.07534974 -3.73233382 ...  2.28801345  0.48994732
  -0.22419151]]


In [1092]:
print(X_test_score_new)

[[-3.8845819   0.65643712 -0.0213289  ...  0.28356137 -0.1871374
   0.17833302]
 [-4.04754212 -0.57698529  0.0609847  ...  0.64010924  0.20487844
  -0.65435265]
 [-3.62914434 -0.32193063 -0.75460868 ...  0.45249467 -0.42689509
   0.50712167]
 ...
 [-4.35941262 -0.51837053 -0.45934686 ...  0.53352694 -0.59794997
   0.33805611]
 [-4.59801611  0.39598001 -0.19308512 ... -0.29584143  2.24377335
   1.09539471]
 [-3.83071435 -0.20358795  0.15808783 ...  0.07165636  0.26965833
  -0.68485615]]


In [1093]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)
print(X_test_score_new.shape, y_test_score.shape)

(8377, 8) (2095, 8)
(8377,) (2095,)
(1000, 8) (1000,)


# 7. Ensemble

In [1094]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score

# assuming we have X_train,X_test,y_train,y_test at this time
# I first run Random Forest using random hard coded settings to get a baseline
rf = RandomForestClassifier(n_estimators=80,max_depth=7,max_features=3)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
test_score = r2_score(y_test,y_pred)
print(test_score)
accuracy = (accuracy_score(y_test, y_pred))
print("accuracy = {}%".format(accuracy))
print("roc_auc_score = {}".format(roc_auc_score(y_test, y_pred)))
print("F1-score = {}".format(f1_score(y_test, y_pred)))

confusion_matrix(y_test, y_pred)

0.9809037494280239
accuracy = 0.9952267303102625%
roc_auc_score = 0.9952874646559849
F1-score = 0.9951876804619826


array([[1051,   10],
       [   0, 1034]])

In [1107]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
# I then use RandomizedSearchCV to find the optimal hyperparameters
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1500, num = 30)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(5, 100, num = 10)]
max_depth.append(None)
min_samples_split = [2, 5, 7, 10]
min_samples_leaf = [2, 5, 7, 10]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

rf = RandomForestClassifier(class_weight='balanced', random_state=42)

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)
rf_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


{'n_estimators': 148,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 57}

In [1108]:
rf = RandomForestClassifier(n_estimators=148,max_depth=57,max_features='sqrt',min_samples_split=5,min_samples_leaf=2,bootstrap=True)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy = (accuracy_score(y_test, y_pred))
print("accuracy = {}%".format(accuracy))

accuracy = 0.9971360381861575%


In [1109]:
# we then use the hyperparameters we found from the RandomizedSearchCV to do a second more thorough check around that range
from sklearn.model_selection import GridSearchCV
param_grid = {
    'bootstrap': [True],
    'max_depth': [65, 68, 71],
    'max_features': ['auto'],
    'min_samples_leaf': [2, 4, 6],
    'min_samples_split': [3, 5, 7],
    'n_estimators': [375, 411, 500]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train)
grid_search.best_params_

Fitting 5 folds for each of 81 candidates, totalling 405 fits


{'bootstrap': True,
 'max_depth': 65,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 375}

In [1110]:
# we then output the results using the optimal hyperparameters to check that our model has improved
rf = RandomForestClassifier(n_estimators=375,max_depth=65,max_features='auto',min_samples_split=5,min_samples_leaf=2,bootstrap=True)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
from sklearn.metrics import r2_score
test_score = r2_score(y_test,y_pred)
test_score

0.9885422496568144

# 8. Neural Network Classifier

In [1095]:
# Neural network
def create_model():
    model = Sequential()
    model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', keras.metrics.Precision(),
                                                                        keras.metrics.Recall(), 
                                                                         keras.metrics.FalsePositives()])
    return model

model = create_model()

model.fit(X_train, y_train, batch_size=20, epochs=50)

y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
accuracy = (accuracy_score(y_test, y_pred))
print("accuracy = {}%".format(accuracy))
print("roc_auc_score = {}".format(roc_auc_score(y_test, y_pred)))
print("F1-score = {}".format(f1_score(y_test, y_pred)))
confusion_matrix(y_test, y_pred)

Epoch 1/50
419/419 [==============================] - 1s 894us/step - loss: 0.2807 - accuracy: 0.8853 - precision_90: 0.8572 - recall_90: 0.9382 - false_positives_90: 217.3881
Epoch 2/50
419/419 [==============================] - 0s 789us/step - loss: 0.0462 - accuracy: 0.9858 - precision_90: 0.9872 - recall_90: 0.9847 - false_positives_90: 30.1857
Epoch 3/50
419/419 [==============================] - 0s 881us/step - loss: 0.0371 - accuracy: 0.9929 - precision_90: 0.9862 - recall_90: 1.0000 - false_positives_90: 27.5119
Epoch 4/50
419/419 [==============================] - 0s 765us/step - loss: 0.0259 - accuracy: 0.9941 - precision_90: 0.9881 - recall_90: 1.0000 - false_positives_90: 23.2500
Epoch 5/50
419/419 [==============================] - 0s 752us/step - loss: 0.0251 - accuracy: 0.9951 - precision_90: 0.9904 - recall_90: 1.0000 - false_positives_90: 20.5119
Epoch 6/50
419/419 [==============================] - 0s 864us/step - loss: 0.0217 - accuracy: 0.9949 - precision_90: 0.9899

419/419 [==============================] - 0s 740us/step - loss: 0.0068 - accuracy: 0.9988 - precision_90: 0.9977 - recall_90: 1.0000 - false_positives_90: 4.9429
Epoch 48/50
419/419 [==============================] - 0s 724us/step - loss: 0.0061 - accuracy: 0.9988 - precision_90: 0.9978 - recall_90: 0.9999 - false_positives_90: 4.9833
Epoch 49/50
419/419 [==============================] - 0s 762us/step - loss: 0.0074 - accuracy: 0.9987 - precision_90: 0.9974 - recall_90: 1.0000 - false_positives_90: 6.3405
Epoch 50/50
419/419 [==============================] - 0s 713us/step - loss: 0.0071 - accuracy: 0.9987 - precision_90: 0.9975 - recall_90: 1.0000 - false_positives_90: 4.8738
accuracy = 0.9966587112171837%
roc_auc_score = 0.9967012252591895
F1-score = 0.9966265060240964


array([[1054,    7],
       [   0, 1034]])

In [1096]:
y_pred = model.predict(X_test_score_new)
y_pred = (y_pred > 0.5)
print(len(y_pred))
accuracy = (accuracy_score(y_test_score, y_pred))
print("accuracy = {}%".format(accuracy))
print("roc_auc_score = {}".format(roc_auc_score(y_test_score, y_pred)))
print("F1-score = {}".format(f1_score(y_test_score, y_pred)))
confusion_matrix(y_test_score, y_pred)

1000
accuracy = 0.915%
roc_auc_score = 0.7200713783497539
F1-score = 0.5812807881773399


array([[856,  15],
       [ 70,  59]])

# 9. Cross-Validate

In [1097]:
pca = PCA(n_components=0.95) # Create an instance of PCA model
pca.fit(X) # Fit X_train to PCA
X_pca = pca.transform(X) # transform training data

In [1098]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.wrappers.scikit_learn import KerasClassifier

kfold = KFold(n_splits=10, random_state=42, shuffle=True)

rf_model_kfold = RandomForestClassifier(n_estimators=80,max_depth=7,max_features=3)

rf_results_kfold = cross_val_score(rf_model_kfold, X_pca, y, cv=kfold)

print("RF Accuracy: %.2f%%" % (rf_results_kfold.mean()*100.0)) 

nn = KerasClassifier(build_fn=create_model, epochs=50, batch_size=20, verbose=0)

nn_results_kfold = cross_val_score(nn, X_pca, y, cv=kfold)

print("NN Accuracy: %.2f%%" % (nn_results_kfold.mean()*100.0))

RF Accuracy: 99.56%
NN Accuracy: 99.72%


# 10. Custom Model

In [1099]:
# Neural network
def create_custom_model(train_data, d1, d2):
    model = Sequential()
    model.add(Dense(25, input_dim=train_data.shape[1], activation='relu'))
    model.add(Dropout(d1))
    model.add(Dense(12, activation='selu'))
    model.add(Dropout(d2))
    model.add(Dense(4, activation='relu'))
#     model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', keras.metrics.Precision(),
                                                                        keras.metrics.Recall(), 
                                                                         keras.metrics.FalsePositives()])
    return model

In [1100]:
model = create_custom_model(X_train, 0.2, 0.1)

model.fit(X_train, y_train, batch_size=20, epochs=75)

y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
accuracy = (accuracy_score(y_test, y_pred))
print("accuracy = {}%".format(accuracy))
print("roc_auc_score = {}".format(roc_auc_score(y_test, y_pred)))
print("F1-score = {}".format(f1_score(y_test, y_pred)))
confusion_matrix(y_test, y_pred)

Epoch 1/75
419/419 [==============================] - 2s 982us/step - loss: 0.4070 - accuracy: 0.8027 - precision_101: 0.7773 - recall_101: 0.8897 - false_positives_101: 351.1857
Epoch 2/75
419/419 [==============================] - 0s 759us/step - loss: 0.0565 - accuracy: 0.9846 - precision_101: 0.9840 - recall_101: 0.9852 - false_positives_101: 37.2119
Epoch 3/75
419/419 [==============================] - 0s 838us/step - loss: 0.0446 - accuracy: 0.9917 - precision_101: 0.9847 - recall_101: 0.9986 - false_positives_101: 31.6810
Epoch 4/75
419/419 [==============================] - 0s 788us/step - loss: 0.0445 - accuracy: 0.9893 - precision_101: 0.9828 - recall_101: 0.9960 - false_positives_101: 33.6905
Epoch 5/75
419/419 [==============================] - 0s 792us/step - loss: 0.0382 - accuracy: 0.9926 - precision_101: 0.9863 - recall_101: 0.9988 - false_positives_101: 25.6048
Epoch 6/75
419/419 [==============================] - 0s 814us/step - loss: 0.0306 - accuracy: 0.9936 - preci

419/419 [==============================] - 0s 801us/step - loss: 0.0185 - accuracy: 0.9968 - precision_101: 0.9939 - recall_101: 0.9999 - false_positives_101: 13.3786
Epoch 47/75
419/419 [==============================] - 0s 831us/step - loss: 0.0158 - accuracy: 0.9970 - precision_101: 0.9941 - recall_101: 1.0000 - false_positives_101: 12.7571
Epoch 48/75
419/419 [==============================] - 0s 856us/step - loss: 0.0104 - accuracy: 0.9977 - precision_101: 0.9959 - recall_101: 0.9996 - false_positives_101: 10.2048
Epoch 49/75
419/419 [==============================] - 0s 813us/step - loss: 0.0185 - accuracy: 0.9962 - precision_101: 0.9931 - recall_101: 0.9994 - false_positives_101: 12.3524
Epoch 50/75
419/419 [==============================] - 0s 874us/step - loss: 0.0184 - accuracy: 0.9957 - precision_101: 0.9932 - recall_101: 0.9985 - false_positives_101: 13.7857
Epoch 51/75
419/419 [==============================] - 0s 798us/step - loss: 0.0186 - accuracy: 0.9962 - precision_10

array([[1056,    5],
       [   0, 1034]])

In [1101]:
# Neural network
def create_custom_model_cv():
    model = Sequential()
    model.add(Dense(25, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(12, activation='selu'))
    model.add(Dropout(0.1))
    model.add(Dense(4, activation='relu'))
#     model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', keras.metrics.Precision(),
                                                                        keras.metrics.Recall(), 
                                                                         keras.metrics.FalsePositives()])
    return model

In [1102]:
kfold = KFold(n_splits=10, random_state=42, shuffle=True)

custom_nn = KerasClassifier(build_fn=create_custom_model_cv, epochs=50, batch_size=20, verbose=0)

custom_nn_results_kfold = cross_val_score(custom_nn, X_pca, y, cv=kfold)

print("Custom NN Accuracy: %.2f%%" % (custom_nn_results_kfold.mean()*100.0))

Custom NN Accuracy: 99.66%


In [1103]:
pca = PCA(n_components=0.95) # Create an instance of PCA model
pca.fit(X) # Fit X_train to PCA
X_pca = pca.transform(X)
X_test_score_kaggle = pca.transform(X_test_score) # transform score test data

In [1104]:
y_pred = [0 for x in range(1000)]

params = [(75, 0.2, 0.1), (10, 0.5, 0.3), (28, 0.1, 0.4), (120, 0.6, 0.1), (50, 0.2, 0.5), (90, 0.2, 0.05),
         (42, 0.25, 0.1), (72, 0.3, 0.15), (25, 0.2, 0.1), (150, 0.2, 0.35)]

for epochs, d1, d2 in params:
    kaggle_model = create_custom_model(X_pca, d1, d2)
    kaggle_model.fit(X_pca, y, batch_size=20, epochs=epochs)

    y_pred_temp = kaggle_model.predict(X_test_score_kaggle)
    y_temp = []
    for i in range(len(y_pred_temp)):
        if y_pred_temp[i] > 0.5:
            y_temp.append(1)
        else:
            y_temp.append(0)
    for i in range(len(y_temp)):
        y_pred[i] += y_temp[i]

Epoch 1/75
524/524 [==============================] - 1s 1ms/step - loss: 0.3442 - accuracy: 0.8264 - precision_112: 0.8644 - recall_112: 0.7681 - false_positives_112: 177.5714
Epoch 2/75
524/524 [==============================] - 1s 1ms/step - loss: 0.0463 - accuracy: 0.9885 - precision_112: 0.9848 - recall_112: 0.9924 - false_positives_112: 41.3581
Epoch 3/75
524/524 [==============================] - 1s 1ms/step - loss: 0.0397 - accuracy: 0.9918 - precision_112: 0.9852 - recall_112: 0.9986 - false_positives_112: 40.3257
Epoch 4/75
524/524 [==============================] - 1s 1ms/step - loss: 0.0303 - accuracy: 0.9937 - precision_112: 0.9888 - recall_112: 0.9990 - false_positives_112: 29.6648
Epoch 5/75
524/524 [==============================] - 1s 1ms/step - loss: 0.0320 - accuracy: 0.9938 - precision_112: 0.9885 - recall_112: 0.9994 - false_positives_112: 29.4990
Epoch 6/75
524/524 [==============================] - 1s 2ms/step - loss: 0.0297 - accuracy: 0.9941 - precision_112: 0.

524/524 [==============================] - 0s 853us/step - loss: 0.0149 - accuracy: 0.9972 - precision_112: 0.9944 - recall_112: 1.0000 - false_positives_112: 14.1638
Epoch 47/75
524/524 [==============================] - 1s 1ms/step - loss: 0.0140 - accuracy: 0.9970 - precision_112: 0.9943 - recall_112: 0.9999 - false_positives_112: 13.5162
Epoch 48/75
524/524 [==============================] - 1s 1ms/step - loss: 0.0120 - accuracy: 0.9967 - precision_112: 0.9943 - recall_112: 0.9991 - false_positives_112: 13.6476
Epoch 49/75
524/524 [==============================] - 1s 1ms/step - loss: 0.0096 - accuracy: 0.9984 - precision_112: 0.9970 - recall_112: 0.9998 - false_positives_112: 9.6743
Epoch 50/75
524/524 [==============================] - 0s 785us/step - loss: 0.0116 - accuracy: 0.9975 - precision_112: 0.9958 - recall_112: 0.9992 - false_positives_112: 12.5257
Epoch 51/75
524/524 [==============================] - 0s 809us/step - loss: 0.0105 - accuracy: 0.9977 - precision_112: 0.99

524/524 [==============================] - 0s 765us/step - loss: 0.0269 - accuracy: 0.9955 - precision_114: 0.9916 - recall_114: 0.9994 - false_positives_114: 25.8133
Epoch 8/28
524/524 [==============================] - 0s 791us/step - loss: 0.0300 - accuracy: 0.9950 - precision_114: 0.9906 - recall_114: 0.9994 - false_positives_114: 24.7010
Epoch 9/28
524/524 [==============================] - 0s 813us/step - loss: 0.0229 - accuracy: 0.9960 - precision_114: 0.9920 - recall_114: 0.9999 - false_positives_114: 23.6076
Epoch 10/28
524/524 [==============================] - 0s 768us/step - loss: 0.0335 - accuracy: 0.9933 - precision_114: 0.9875 - recall_114: 0.9995 - false_positives_114: 31.1219
Epoch 11/28
524/524 [==============================] - 0s 802us/step - loss: 0.0277 - accuracy: 0.9951 - precision_114: 0.9916 - recall_114: 0.9989 - false_positives_114: 23.6362
Epoch 12/28
524/524 [==============================] - 0s 771us/step - loss: 0.0289 - accuracy: 0.9949 - precision_114:

524/524 [==============================] - 0s 898us/step - loss: 0.0286 - accuracy: 0.9941 - precision_115: 0.9900 - recall_115: 0.9983 - false_positives_115: 26.2190
Epoch 25/120
524/524 [==============================] - 0s 826us/step - loss: 0.0270 - accuracy: 0.9941 - precision_115: 0.9915 - recall_115: 0.9969 - false_positives_115: 23.4610
Epoch 26/120
524/524 [==============================] - 0s 859us/step - loss: 0.0321 - accuracy: 0.9937 - precision_115: 0.9895 - recall_115: 0.9977 - false_positives_115: 27.0019
Epoch 27/120
524/524 [==============================] - 0s 824us/step - loss: 0.0248 - accuracy: 0.9946 - precision_115: 0.9916 - recall_115: 0.9975 - false_positives_115: 20.8419
Epoch 28/120
524/524 [==============================] - 0s 849us/step - loss: 0.0255 - accuracy: 0.9941 - precision_115: 0.9916 - recall_115: 0.9965 - false_positives_115: 24.8248
Epoch 29/120
524/524 [==============================] - 0s 820us/step - loss: 0.0322 - accuracy: 0.9932 - precisi

524/524 [==============================] - 0s 822us/step - loss: 0.0267 - accuracy: 0.9934 - precision_115: 0.9916 - recall_115: 0.9953 - false_positives_115: 24.4114
Epoch 70/120
524/524 [==============================] - 0s 883us/step - loss: 0.0248 - accuracy: 0.9948 - precision_115: 0.9925 - recall_115: 0.9970 - false_positives_115: 20.6857
Epoch 71/120
524/524 [==============================] - 1s 1ms/step - loss: 0.0323 - accuracy: 0.9934 - precision_115: 0.9900 - recall_115: 0.9970 - false_positives_115: 22.6419
Epoch 72/120
524/524 [==============================] - 1s 1ms/step - loss: 0.0251 - accuracy: 0.9943 - precision_115: 0.9909 - recall_115: 0.9975 - false_positives_115: 22.1219
Epoch 73/120
524/524 [==============================] - 0s 875us/step - loss: 0.0212 - accuracy: 0.9960 - precision_115: 0.9934 - recall_115: 0.9988 - false_positives_115: 19.6952
Epoch 74/120
524/524 [==============================] - 0s 928us/step - loss: 0.0171 - accuracy: 0.9957 - precision_1

...

524/524 [==============================] - 0s 761us/step - loss: 0.0175 - accuracy: 0.9967 - precision_119: 0.9939 - recall_119: 0.9996 - false_positives_119: 17.4362
Epoch 39/72
524/524 [==============================] - 0s 819us/step - loss: 0.0135 - accuracy: 0.9968 - precision_119: 0.9940 - recall_119: 0.9998 - false_positives_119: 17.6933
Epoch 40/72
524/524 [==============================] - 0s 910us/step - loss: 0.0185 - accuracy: 0.9959 - precision_119: 0.9929 - recall_119: 0.9989 - false_positives_119: 17.2762
Epoch 41/72
524/524 [==============================] - 0s 892us/step - loss: 0.0120 - accuracy: 0.9979 - precision_119: 0.9959 - recall_119: 1.0000 - false_positives_119: 13.1524
Epoch 42/72
524/524 [==============================] - 1s 965us/step - loss: 0.0225 - accuracy: 0.9943 - precision_119: 0.9901 - recall_119: 0.9985 - false_positives_119: 21.6914
Epoch 43/72
524/524 [==============================] - 0s 896us/step - loss: 0.0161 - accuracy: 0.9961 - precision_11

524/524 [==============================] - 0s 768us/step - loss: 0.0233 - accuracy: 0.9948 - precision_120: 0.9903 - recall_120: 0.9994 - false_positives_120: 26.1505
Epoch 12/25
524/524 [==============================] - 0s 833us/step - loss: 0.0249 - accuracy: 0.9945 - precision_120: 0.9899 - recall_120: 0.9991 - false_positives_120: 25.2343
Epoch 13/25
524/524 [==============================] - 0s 783us/step - loss: 0.0205 - accuracy: 0.9957 - precision_120: 0.9918 - recall_120: 0.9998 - false_positives_120: 24.1162
Epoch 14/25
524/524 [==============================] - 0s 800us/step - loss: 0.0221 - accuracy: 0.9946 - precision_120: 0.9895 - recall_120: 0.9999 - false_positives_120: 24.0038
Epoch 15/25
524/524 [==============================] - 0s 738us/step - loss: 0.0215 - accuracy: 0.9945 - precision_120: 0.9896 - recall_120: 0.9995 - false_positives_120: 25.3029
Epoch 16/25
524/524 [==============================] - 0s 777us/step - loss: 0.0202 - accuracy: 0.9957 - precision_12

524/524 [==============================] - 0s 832us/step - loss: 0.0251 - accuracy: 0.9959 - precision_121: 0.9935 - recall_121: 0.9981 - false_positives_121: 17.4419
Epoch 31/150
524/524 [==============================] - 0s 794us/step - loss: 0.0192 - accuracy: 0.9973 - precision_121: 0.9947 - recall_121: 0.9999 - false_positives_121: 16.5010
Epoch 32/150
524/524 [==============================] - 0s 818us/step - loss: 0.0233 - accuracy: 0.9962 - precision_121: 0.9924 - recall_121: 1.0000 - false_positives_121: 19.4362
Epoch 33/150
524/524 [==============================] - 0s 808us/step - loss: 0.0258 - accuracy: 0.9958 - precision_121: 0.9932 - recall_121: 0.9984 - false_positives_121: 18.2648
Epoch 34/150
524/524 [==============================] - 0s 780us/step - loss: 0.0173 - accuracy: 0.9972 - precision_121: 0.9950 - recall_121: 0.9996 - false_positives_121: 15.5619
Epoch 35/150
524/524 [==============================] - 0s 744us/step - loss: 0.0184 - accuracy: 0.9967 - precisi

524/524 [==============================] - 0s 808us/step - loss: 0.0148 - accuracy: 0.9972 - precision_121: 0.9956 - recall_121: 0.9988 - false_positives_121: 11.9333
Epoch 76/150
524/524 [==============================] - 0s 820us/step - loss: 0.0125 - accuracy: 0.9980 - precision_121: 0.9970 - recall_121: 0.9990 - false_positives_121: 9.7390
Epoch 77/150
524/524 [==============================] - 0s 821us/step - loss: 0.0124 - accuracy: 0.9974 - precision_121: 0.9957 - recall_121: 0.9993 - false_positives_121: 11.9638
Epoch 78/150
524/524 [==============================] - 0s 817us/step - loss: 0.0146 - accuracy: 0.9971 - precision_121: 0.9943 - recall_121: 1.0000 - false_positives_121: 14.4724
Epoch 79/150
524/524 [==============================] - 0s 859us/step - loss: 0.0170 - accuracy: 0.9971 - precision_121: 0.9943 - recall_121: 0.9999 - false_positives_121: 14.2895
Epoch 80/150
524/524 [==============================] - 0s 822us/step - loss: 0.0092 - accuracy: 0.9983 - precisio

524/524 [==============================] - 0s 781us/step - loss: 0.0114 - accuracy: 0.9978 - precision_121: 0.9963 - recall_121: 0.9993 - false_positives_121: 10.3124
Epoch 121/150
524/524 [==============================] - 0s 803us/step - loss: 0.0090 - accuracy: 0.9985 - precision_121: 0.9969 - recall_121: 1.0000 - false_positives_121: 9.8400
Epoch 122/150
524/524 [==============================] - 0s 798us/step - loss: 0.0077 - accuracy: 0.9987 - precision_121: 0.9976 - recall_121: 0.9998 - false_positives_121: 8.1543
Epoch 123/150
524/524 [==============================] - 0s 751us/step - loss: 0.0140 - accuracy: 0.9970 - precision_121: 0.9951 - recall_121: 0.9989 - false_positives_121: 13.1867
Epoch 124/150
524/524 [==============================] - 0s 846us/step - loss: 0.0151 - accuracy: 0.9972 - precision_121: 0.9950 - recall_121: 0.9994 - false_positives_121: 12.6381
Epoch 125/150
524/524 [==============================] - 0s 795us/step - loss: 0.0098 - accuracy: 0.9978 - prec

In [1105]:
for i in range(len(y_pred)):
    y_pred[i] = y_pred[i]/10
print(y_pred)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.4, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.4, 0.0, 1.0, 0.0, 0.6, 0.0, 0.0, 0.0, 0.3, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9, 0.0, 0.2, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 1.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.3, 0.6, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.3, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,

In [1106]:
index = np.array([int(x) for x in range(83414, 84414)])
kaggle_df = pd.DataFrame(index, columns=['ID'])

y_pred_temp = []
for i in range(len(y_pred)):
    if y_pred[i] > 0.5:
        y_pred_temp.append(1)
    else:
        y_pred_temp.append(0)
y_pred_temp = np.array(y_pred_temp)
kaggle_df["Predicted"] = y_pred_temp

print(np.mean(y_pred_temp))

kaggle_df

0.098


,ID,Predicted
0,83414,0
1,83415,0
2,83416,0
3,83417,0
4,83418,0
...,...,...
995,84409,0
996,84410,0
997,84411,0
998,84412,0


In [1000]:
kaggle_df.to_csv("/Users/shrenik/Desktop/UCLA/Year-3/labels_1.csv", index=False, header=True)